In [4]:
from decimal import Decimal, getcontext, ROUND_HALF_UP
import ipywidgets as widgets
from IPython.display import display, clear_output
import re

getcontext().prec = 28
getcontext().rounding = ROUND_HALF_UP

MIN_VALUE = Decimal('-1000000000000.000000')
MAX_VALUE = Decimal('1000000000000.000000')

def parse_number(input_str: str) -> Decimal:
    """Строгий парсинг с правильной проверкой пробелов как разделителей тысяч."""
    s = input_str.strip()
    
    if not s:
        raise ValueError("Пустой ввод")

    no_spaces = s.replace(' ', '')
    normalized_for_check = no_spaces.replace(',', '.')
    try:
        Decimal(normalized_for_check)
    except:
        raise ValueError("Некорректные символы в числе (буквы, лишние знаки и т.п.)")
    
    if re.search(r'[+-].*[+-]', no_spaces):
        raise ValueError("Некорректный знак в числе (например, '0.0-1')")
    
    if '.' in no_spaces and ',' in no_spaces:
        raise ValueError("Использовано одновременно точка и запятая")
    
    frac_sep = '.' if '.' in s else ',' if ',' in s else None
    if frac_sep:
        original_int_part = s.split(frac_sep)[0].strip()
    else:
        original_int_part = s.strip()
    
    if original_int_part.startswith(('-', '+')):
        original_int_part = original_int_part[1:].lstrip()  
    
    if ' ' not in original_int_part:
        final_normalized = no_spaces.replace(',', '.')
        return Decimal(final_normalized)

    groups = [g for g in original_int_part.split(' ') if g]  
    
    if not groups:
        raise ValueError("Неверное расположение пробелов")
    
    for group in groups[1:]:
        if len(group) != 3 or not group.isdigit():
            raise ValueError("Пробелы должны разделять группы строго по 3 цифры")
    
    if not (1 <= len(groups[0]) <= 3) or not groups[0].isdigit():
        raise ValueError("Самая левая группа цифр должна иметь от 1 до 3 цифр")

    final_normalized = no_spaces.replace(',', '.')
    return Decimal(final_normalized)

def format_integer(int_str: str) -> str:
    """Форматирует целую часть с пробелами по 3 цифры."""
    if not int_str or int_str == '-':
        return '0'
    if int_str.startswith('-'):
        sign = '-'
        int_str = int_str[1:]
    else:
        sign = ''
    
    formatted = ''
    for i, digit in enumerate(reversed(int_str)):
        if i > 0 and i % 3 == 0:
            formatted = ' ' + formatted
        formatted = digit + formatted
    
    return sign + (formatted or '0')

def format_result(value: Decimal) -> str:
    """Форматирует результат с пробелами и до 6 знаков после точки без trailing zeros."""
    rounded = value.quantize(Decimal('0.000000'), rounding=ROUND_HALF_UP)
    s = f"{rounded:f}"
    
    if '.' in s:
        int_part, frac_part = s.split('.')
        frac_part = frac_part.rstrip('0')
        if frac_part:
            return format_integer(int_part) + '.' + frac_part
        else:
            return format_integer(int_part)
    else:
        return format_integer(s)

label_info = widgets.Label(
    value="Крепская Анна Валерьевна, Курс: 4, Группа: 4, Год: 2025"
)

input_a = widgets.Text(value='0.0', description='Число A:', continuous_update=False)
input_b = widgets.Text(value='0.0', description='Число B:', continuous_update=False)

operation = widgets.Dropdown(
    options=['Сложение', 'Вычитание', 'Умножение', 'Деление'],
    value='Сложение',
    description='Операция:'
)

button_calculate = widgets.Button(description='Вычислить', button_style='success')
output_result = widgets.Output()

def calculate(_):
    with output_result:
        clear_output()
        try:
            a = parse_number(input_a.value)
            b = parse_number(input_b.value)
            
            if operation.value == 'Сложение':
                result = a + b
            elif operation.value == 'Вычитание':
                result = a - b
            elif operation.value == 'Умножение':
                result = a * b
            elif operation.value == 'Деление':
                if b == 0:
                    print("Ошибка: деление на ноль невозможно")
                    return
                result = a / b
            
            if result < MIN_VALUE or result > MAX_VALUE:
                print("Переполнение: результат вне диапазона (±1 000 000 000 000.000000)")
            else:
                formatted = format_result(result)
                print(f"Результат: {formatted}")
                
        except ValueError as e:
            print(f"Ошибка ввода: {e}")
        except Exception as e:
            print(f"Неизвестная ошибка: {str(e)}")

button_calculate.on_click(calculate)

display(label_info)
display(input_a, input_b)
display(operation)
display(button_calculate)
display(output_result)

Label(value='Крепская Анна Валерьевна, Курс: 4, Группа: 4, Год: 2025')

Text(value='0.0', continuous_update=False, description='Число A:')

Text(value='0.0', continuous_update=False, description='Число B:')

Dropdown(description='Операция:', options=('Сложение', 'Вычитание', 'Умножение', 'Деление'), value='Сложение')

Button(button_style='success', description='Вычислить', style=ButtonStyle())

Output()